In [7]:
import momepy

/opt/miniconda3/envs/knupu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import geopandas as gpd
from collections import Counter

In [2]:
nodepath = "../data/raw/folkersma_digital/2023-08-01 Cycling network Denmark shapefiles/node.shp"
edgepath = "../data/raw/folkersma_digital/2023-08-01 Cycling network Denmark shapefiles/stretch.shp"

In [14]:
nodes = gpd.read_file(nodepath)
edges = gpd.read_file(edgepath)

In [15]:
nodes.head()

,id,nodenumber,comment,region,status,uuid,ismain,refmain,deadend,geometry
0,1009,10.0,NaN,RingkÃ¸bing-Skjern,Concept,vk1009,1,NaN,0,POINT (8.17250 56.17202)
1,5078,90.0,NaN,Vordingborg,Concept,vk5078,0,5076.0,0,POINT (11.87677 55.03425)
2,5119,14.0,NaN,Varde,Concept,vk5119,0,5008.0,0,POINT (8.61188 55.72085)
3,5125,61.0,NaN,Aabenraa,Concept,vk5125,0,5150.0,0,POINT (9.43320 55.11575)
4,5129,59.0,NaN,SÃ¸nderborg,Concept,vk5129,0,1401.0,0,POINT (9.87048 54.98480)


In [16]:
assert all([n[0:2]=="vk" for n in nodes.uuid])
assert all(nodes.id == nodes.uuid.apply(lambda x: int(x[2:])))
# id is unique; uuid is just "vk" + id;

In [17]:
Counter(nodes.ismain) # 1362 parent and 700 child nodes

Counter({1: 1362, 0: 700})

In [18]:
assert all(nodes[nodes.ismain  == 1]["refmain"].isna()) # all main nodes have no "main ref"

In [19]:
# a half of non-main nodes have no main ref
sum(nodes[nodes.ismain == 0]["refmain"].isna())

332

In [20]:
edges.head()

,id,rating,comment,length,privatenot,surfacenot,oneway,region,uuid,status,geometry
0,1387,1,NaN,0.495024,0,0,1,Varde,l1387,Concept,"LINESTRING (8.45851 55.63194, 8.45847 55.63192..."
1,1989,1,NaN,0.078030,0,0,1,SÃ¸nderborg,l1989,Concept,"LINESTRING (9.99485 54.87964, 9.99485 54.87965..."
2,7224,1,NaN,5.034481,0,0,1,Aabenraa,l7224,Concept,"LINESTRING (9.29901 55.07036, 9.29900 55.07038..."
3,5984,1,NaN,0.545476,0,0,0,SÃ¸nderborg,l5984,Concept,"LINESTRING (9.78557 55.05005, 9.78574 55.04976..."
4,6057,1,NaN,3.023506,0,1,0,SÃ¸nderborg,l6057,Concept,"LINESTRING (9.66361 54.86392, 9.66363 54.86392..."


In [9]:
edges.to_crs("EPSG:25832",inplace=True)

In [10]:
G_primal = momepy.gdf_to_nx(edges, approach="primal")

In [11]:
G_primal

In [21]:
mome_nodes, mome_edges = momepy.nx_to_gdf(G_primal)

In [22]:
len(mome_nodes)

2578

In [23]:
len(nodes)

2062

In [24]:
mome_nodes.to_file("../data/processed/mome_test_nodes.gpkg")

In [ ]:
# Conclusion - topo issues - momepy creates additional nodes, not properly snapped

# Still necessary with snapping etc. Also some nodes missing at intersections (maybe small overshoot edges that should not be there?)

# Get parent - child info: Snap/join our nodes to Folkersma

# Transfer info